## <font color='Blue'>Introduction</font>
###  In this exercise, we run simulation in a 2D single layer two-phase model.
* The realistic formation heterogeneity is used for permeability map. 

## <font color='blue'>The objectives:</font>
1. Introduce custom <font color='red'>'Model'</font> class based on 
 * Class [DartsModel](https://gitlab.com/open-darts/open-darts/-/blob/development/darts-package/darts/models/darts_model.py)  with base model capabilities
 * Class [StructReservoir](https://gitlab.com/open-darts/open-darts/-/blob/development/darts-package/darts/models/reservoirs/struct_reservoir.py) with structured reservoir
 * Class [SuperPhysics](https://gitlab.com/open-darts/open-darts/-/blob/development/darts-package/darts/models/physics_sup/physics_comp_sup.py) for basic physics where super engine is used
 * Class [PropertyContainer](https://gitlab.com/open-darts/open-darts/-/blob/development/darts-package/darts/models/physics_sup/property_container.py) for allocating property evaluator needed in the simulation
 * Class [Properties](Egg/properties_dead_oil.py) for dead oil properties in this practice, such as density, viscosity and relative permeability. It should be changed based on the investigated problem.
2. Use run_python procedure to control run from the python script
3. Introduce wells and time-dependent well controls
4. Redefine physical properties and run simulation with custom-defined property.

## <font color='Blue'>Let's start!</font>


<img style="float: left;" src="slides/Slide33.JPG" width="60%">

In [ ]:
from darts.models.reservoirs.struct_reservoir import StructReservoir
from darts.models.darts_model import DartsModel
from darts.models.physics_sup.physics_comp_sup import Compositional as SuperPhysics
from darts.models.physics_sup.property_container import *
from Egg.properties_dead_oil import *
from darts.tools.keyword_file_tools import load_single_keyword
from darts.engines import redirect_darts_output
import numpy as np
redirect_darts_output('log.txt')

<img style="float: left;" src="slides/Slide34.JPG" width="60%">

#### Brief Introduction of model inheritance
* Here create the <font color='red'>'Model' </font>  class, which inherits from <font color='red'>DartsModel</font> (the base class).
* It keeps all the functionalities of <font color='red'>DartsModel</font> and can also be extended to add more functionalities.
* If a function is redefined in subclass, the function in base class with identical name will be overridden.

In [ ]:
class Model(DartsModel):
    def __init__(self):
        # call base class constructor
        super().__init__()

        # measure time spend on reading/initialization
        self.timer.node["initialization"].start()

        # create reservoir
        self.nx = 60
        self.ny = 60
        self.nz = 1
        
        self.dx = 8
        self.dy = 8
        self.dz = 4

        self.permx = load_single_keyword('Egg/data.in','PERMX')
        self.actnum = load_single_keyword('Egg/data.in','ACTNUM')            
        
        self.permy = self.permx
        self.permz = 0.1 * self.permx
        self.poro = 0.2
        self.depth = 4000

        # run discretization
        self.reservoir = StructReservoir(self.timer, nx=self.nx, ny=self.ny, nz=self.nz, dx=self.dx, dy=self.dy,
                                         dz=self.dz, permx=self.permx, permy=self.permy, permz=self.permz,
                                         poro=self.poro, depth=self.depth,actnum=self.actnum)
        
        self.well_init()
               
        """Physical properties"""
        self.pvt = 'Egg/physics.in'
        self.zero = 1e-13
        self.property_container = model_properties(phases_name=['water', 'oil'], components_name=['w', 'o'], 
                                                   pvt=self.pvt, min_z=self.zero/10)

        # Define property evaluators based on custom properties
        self.flash_ev = []
        self.property_container.density_ev = dict([('water', DensityWat(self.pvt)),
                                                   ('oil', DensityOil(self.pvt))])
        self.property_container.viscosity_ev = dict([('water', ViscoWat(self.pvt)),
                                                     ('oil', ViscoOil(self.pvt))])
        self.property_container.rel_perm_ev = dict([('water', WatRelPerm(self.pvt)),
                                                    ('oil', OilRelPerm(self.pvt))])
        self.property_container.capillary_pressure_ev = CapillarypressurePcow(self.pvt)

        self.property_container.rock_compress_ev = RockCompactionEvaluator(self.pvt)

        # create physics
        self.thermal = 0
        self.physics = SuperPhysics(self.property_container, self.timer, n_points=400, min_p=0, max_p=1000,
                                     min_z=self.zero, max_z=1 - self.zero, thermal=self.thermal)

        self.params.first_ts = 0.01
        self.params.mult_ts = 2
        self.params.max_ts = 10
        self.params.tolerance_newton = 1e-2
        self.params.tolerance_linear = 1e-4

        self.inj = [0.999]

        self.runtime = 100

        self.timer.node["initialization"].stop()
        
    def well_init(self):
        well_diam = 0.2
        well_rad = well_diam/2
        
        wells = np.loadtxt('Egg/wells.in')
        for i, wcoord in enumerate(wells):            
            if i < 8:
                self.reservoir.add_well("INJ" + str(i+1))
            else:
                self.reservoir.add_well("PRD" + str(i+1 - 8))
                    
            self.reservoir.add_perforation(self.reservoir.wells[-1], int(wcoord[0]), int(wcoord[1]), 1, 
                                           well_radius=well_rad, multi_segment=False)
            self.reservoir.inj_wells = [self.reservoir.wells[-1]]
        
    def set_initial_conditions(self):
        self.physics.set_uniform_initial_conditions(self.reservoir.mesh, uniform_pressure=400,
                                                    uniform_composition=[2e-2])

    def set_boundary_conditions(self):
        for i, w in enumerate(self.reservoir.wells):
            if "INJ" in w.name:
                w.control = self.physics.new_rate_inj(200, self.inj, 0)
                w.constraint = self.physics.new_bhp_inj(450, self.inj)
            else:
                w.control = self.physics.new_bhp_prod(390)

<img style="float: left;" src="slides/Slide45.JPG" width="60%">

<img style="float: left;" src="slides/Slide46.JPG" width="60%">

<img style="float: left;" src="slides/Slide35.JPG" width="60%">

In [ ]:
# evaluate all properties which are needed in the simulation
class model_properties(property_container):
    def __init__(self, phases_name, components_name, pvt, min_z=1e-11):
        # Call base class constructor
        self.nph = len(phases_name)
        Mw = np.ones(self.nph)
        super().__init__(phases_name, components_name, Mw, min_z)
        self.x = np.zeros((self.nph, self.nc))
        self.pvt = pvt
        self.surf_dens = get_table_keyword(self.pvt, 'DENSITY')[0]
        self.surf_oil_dens = self.surf_dens[0]
        self.surf_wat_dens = self.surf_dens[1]

    def evaluate(self, state):
        """
        Class methods which evaluates the state operators for the element based physics
        :param state: state variables [pres, comp_0, ..., comp_N-1]
        :param values: values of the operators (used for storing the operator values)
        :return: updated value for operators, stored in values
        """
        # Composition vector and pressure from state:
        vec_state_as_np = np.asarray(state)
        pressure = vec_state_as_np[0]

        zc = np.append(vec_state_as_np[1:], 1 - np.sum(vec_state_as_np[1:]))

        self.clean_arrays()
        # two-phase flash - assume water phase is always present and water component last
        for i in range(self.nph):
            self.x[i, i] = 1

        ph = [0, 1]

        for j in ph:
            M = 0
            # molar weight of mixture
            for i in range(self.nc):
                M += self.Mw[i] * self.x[j][i]
            self.dens[j] = self.density_ev[self.phases_name[j]].evaluate(state)  # output in [kg/m3]
            self.dens_m[j] = self.dens[j] / M
            self.mu[j] = self.viscosity_ev[self.phases_name[j]].evaluate(state)  # output in [cp]

        self.nu = zc
        self.compute_saturation(ph)
        
        # when evaluate rel-perm based on the table, we only need water saturation to interpolate both phases saturation
        for j in ph:
            self.kr[j] = self.rel_perm_ev[self.phases_name[j]].evaluate(self.sat[0])
            self.pc[j] = 0

        return self.sat, self.x, self.dens, self.dens_m, self.mu, self.kr, self.pc, ph

    def evaluate_at_cond(self, pressure, zc):

        self.sat[:] = 0

        state = value_vector([1, 0])

        ph = [0, 1]
        for j in ph:
            self.dens_m[j] = self.density_ev[self.phases_name[j]].evaluate(state)

        self.dens_m = [self.surf_wat_dens, self.surf_oil_dens]  # to match DO based on PVT

        self.nu = zc
        self.compute_saturation(ph)

        return self.sat, self.dens_m

<img style="float: left;" src="slides/Slide36.JPG" width="60%">

In [ ]:
m = Model()
m.init()
m.run_python()
m.print_timers()

<img style="float: left;" src="slides/Slide37.JPG" width="60%">

In [ ]:
%matplotlib inline
import pandas as pd
time_data = pd.DataFrame.from_dict(m.physics.engine.time_data)
# wirte timedata to output file
time_data.to_pickle("darts_time_data.pkl")
# write timedata to excel file
writer = pd.ExcelWriter('time_data.xlsx')
time_data.to_excel(writer, 'Sheet1')
writer.save()

In [ ]:
td = pd.read_pickle("darts_time_data.pkl")
from darts.tools.plot_darts import *
plot_water_rate_darts('INJ1', time_data)
plot_bhp_darts('INJ1', time_data)
plot_oil_rate_darts('PRD1', time_data)
plt.show()

<img style="float: left;" src="slides/Slide38.JPG" width="60%">

In [ ]:
import matplotlib.pyplot as plt
plt.figure(num=1, figsize=(12, 4))
plt.subplot(121)
perm = np.array(m.permx)
plt.pcolor(perm.reshape(m.ny, m.nx))
plt.title('perm map')
plt.colorbar()

plt.subplot(122)
act = np.array(m.actnum)
plt.pcolor(act.reshape(m.ny, m.nx))
plt.title('Active cells map')
plt.colorbar()

### Next we process the active grid, prepare plotting and plot pressure

<img style="float: left;" src="slides/Slide42.JPG" width="60%">

In [ ]:
import matplotlib.pyplot as plt

# process active grid
X = np.array(m.physics.engine.X, copy=False)
nb = m.nx * m.ny
nb = np.count_nonzero(m.actnum)

p = -np.ones(m.nx*m.ny)
z = -np.ones(m.nx*m.ny)
s = -np.ones(m.nx*m.ny)

p[act>0] = X[0:2*nb:2]
z[act>0] = X[1:2*nb:2]

plt.figure(num=2, figsize=(12,4))
plt.subplot(121)
plt.pcolor(p.reshape(m.ny, m.nx),cmap='jet')
plt.clim([390,max(p)])
plt.colorbar()

plt.subplot(122)
plt.pcolor(z.reshape(m.ny, m.nx),cmap='jet')
plt.colorbar()
plt.clim([0,max(z)])
